In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow import keras
from tensorflow.keras import layers
from keras import models, layers
import tqdm
from PIL import Image

import os

In [ ]:
PATH = "/kaggle/input/challenges-in-representation-learning-facial-expression-recognition-challenge"

data = pd.read_csv(os.path.join(PATH, "icml_face_data.csv"))

emotions = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

In [ ]:
print(data)

In [ ]:
# Function to parse data into right format
# Output: Image in right shaped and normalized + labels
def parse_data(data):
    image_array = np.zeros(shape=(len(data), 48, 48, 1))
    image_label = np.array(list(map(int, data['emotion'])))
    
    for i, row in enumerate(data.index):
        image = np.fromstring(data.loc[row, ' pixels'], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48, 1))
        image_array[i] = image
        
    return image_array, image_label

# Splitting the data into train, validation and testing set thanks to Usage column
train_imgs, train_lbls = parse_data(data[data[" Usage"] == "Training"])
val_imgs, val_lbls = parse_data(data[data[" Usage"] == "PrivateTest"])
test_imgs, test_lbls = parse_data(data[data[" Usage"] == "PublicTest"])

In [ ]:
print("train shape", np.shape(train_imgs))
print("validation shape", np.shape(val_imgs))
print("validatio shape", np.shape(val_imgs))

In [ ]:
print(train_imgs)

Image files generation

In [ ]:
import os, shutil 
os.mkdir("/kaggle/working/imgs")
data = np.array(train_imgs[:5])
i = 0
for px_map in data:
    i = i + 1
    px_map = np.reshape(px_map, (48, 48))
    image = Image.fromarray(px_map)
    image = image.convert('RGB')
    image.save('/kaggle/working/imgs/'+str(i)+'.bmp')

https://scikit-learn.org/stable/modules/cross_validation.html

MLP model

In [ ]:
# Building a MLP model based on LeNet architecture 

model_mlp = keras.Sequential()

model_mlp.add(layers.Flatten(input_shape=(48, 48, 1)))

model_mlp.add(layers.Dense(units=120, activation='relu'))

model_mlp.add(layers.Dense(units=84, activation='relu'))

model_mlp.add(layers.Dense(units=7, activation = 'softmax'))

model_mlp.compile(loss=keras.losses.SparseCategoricalCrossentropy(), optimizer=keras.optimizers.Adam(lr=1e-3), metrics=['accuracy'])

model_mlp.summary()

In [ ]:
# Training the model, and validating
model_mlp.fit(train_imgs, train_lbls, 
          epochs=5, batch_size=32, 
          validation_data=(val_imgs, val_lbls), verbose=1)

CNN model

In [ ]:
model_cnn = keras.Sequential()

model_cnn.add(layers.Conv2D())

model_cnn.add(layers.MaxPooling())

model_cnn.add(layers.Flatten())

model_cnn.add(layers.Dense(units=84, activation='relu'))

model_cnn.add(layers.Dense(units=7, activation = 'softmax'))

model_cnn.compile(loss=keras.losses.SparseCategoricalCrossentropy(), optimizer=keras.optimizers.Adam(lr=1e-3), metrics=['accuracy'])

model_cnn.summary()

In [ ]:
model_cnn.fit()